In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
from scipy.signal import resample, find_peaks
from scipy.io import loadmat
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import pickle
import time

# from sklearn import preprocessing ##importing for normalization
# from sklearn.decomposition import PCA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler, minmax_scale, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.multioutput import MultiOutputRegressor
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['legend.frameon'] = True
# plt.rcParams['figure.figsize'] = (8, 8)
%matplotlib widget

In [2]:
with open('train_resample.pickle', 'rb') as f:
    train_resample = pickle.load(f)

with open('train_norm_labels.pickle', 'rb') as f:
    train_norm_labels = pickle.load(f)

with open('training_labels.pickle', 'rb') as f:
    training_labels = pickle.load(f)

with open('exp_val_resample.pickle', 'rb') as f:
    exp_val_resample = pickle.load(f)

with open('exp_val_norm_labels.pickle', 'rb') as f:
    exp_val_labels = pickle.load(f)

X = train_resample
y = training_labels

exp_val_X = exp_val_resample
exp_val_y = exp_val_labels

scalar = MinMaxScaler()
y_norm = scalar.fit_transform(training_labels)

print(np.asarray(X).shape, np.asarray(y).shape)

# create split data from the normalized data
size = 0.1
x_train, x_test, y_train, y_test = train_test_split(X, y_norm, shuffle=True, test_size=size)
print(f'Length of training data - {np.asarray(x_train).shape} length of labels - {np.asarray(y_train).shape} \nLength of test data - {np.asarray(x_test).shape} and length of labels - {np.asarray(y_test).shape}')

(925, 100, 4) (925, 2)
Length of training data - (832, 100, 4) length of labels - (832, 2) 
Length of test data - (93, 100, 4) and length of labels - (93, 2)


In [3]:
x_train_flat = []
for i in range(len(x_train)):    
    x_dummy = np.asarray(x_train[i]).flatten()
    x_train_flat.append(x_dummy)
    
x_train_flat = np.asarray(x_train_flat)

x_test_flat = []
for i in range(len(x_test)):    
    x_dummy = np.asarray(x_test[i]).flatten()
    x_test_flat.append(x_dummy)
    
x_test_flat = np.asarray(x_test_flat)

exp_val_X_flat = []
for i in range(len(exp_val_X)):
    x_dummy = np.asarray(exp_val_X[i]).flatten()
    exp_val_X_flat.append(x_dummy)

exp_val_X_flat = np.asarray(exp_val_X_flat)

print(np.asarray(x_train_flat).shape, np.asarray(y_train).shape)
print(x_test_flat.shape, y_test.shape)
print(exp_val_X_flat.shape, exp_val_y.shape)

(832, 400) (832, 2)
(93, 400) (93, 2)
(24, 400) (24, 2)


In [4]:
knn = KNeighborsRegressor(n_neighbors=5, weights='distance')
knn.fit(x_train_flat, y_train)

KNeighborsRegressor(weights='distance')

In [5]:
y_pred = knn.predict(x_test_flat)

y_pred = scalar.inverse_transform(y_pred)
y_test = scalar.inverse_transform(y_test)

y_test-y_pred

array([[ 8.88713219e-01,  1.83278347e+00],
       [-4.37480466e+00,  4.37480313e+00],
       [-1.77287518e+00,  9.22126809e-01],
       [-2.73039806e-01, -6.86830652e-01],
       [ 7.67068902e-01,  4.80819938e-02],
       [-9.01810045e-01,  2.75936001e-02],
       [-5.07840495e+01, -5.07808621e+01],
       [ 8.99956766e-01, -3.71819595e-02],
       [ 1.47342084e+00, -1.37807416e+00],
       [ 1.63273430e+00, -7.43236571e-01],
       [ 2.88573015e+00, -1.59601418e+01],
       [-1.78017254e-01, -1.76508270e+00],
       [ 1.34104778e+01, -4.94894605e-02],
       [ 4.85981177e+00, -4.12674303e+00],
       [ 8.77159197e-01, -1.07273421e+00],
       [-2.46943999e+00,  4.05449932e+00],
       [-8.84684840e-01, -1.59233311e+00],
       [-8.58366438e-02,  1.36551103e+00],
       [-7.53719216e-01,  8.54466335e-01],
       [-1.61355797e+00, -7.50454994e-01],
       [ 8.42024672e-01, -7.20608012e-01],
       [-4.56860299e-01, -1.09114360e-02],
       [ 7.72743232e-01,  1.73442174e+00],
       [-7.

In [6]:
y_pred_exp = knn.predict(exp_val_X_flat)
y_pred_exp = scalar.inverse_transform(y_pred_exp)
exp_val_y = scalar.inverse_transform(exp_val_y)

exp_val_y-y_pred_exp

array([[-34.60210813, -31.63818693],
       [-34.73297794, -31.64591079],
       [-26.94172669, -55.97825994],
       [-17.06847129, -16.97882429],
       [-10.38875905,  13.03068487],
       [ -9.50346812,  -8.77040225],
       [-11.50612416,  11.07668314],
       [-11.41860212,  11.98863275],
       [-32.41854898,  23.80030127],
       [-28.94934003,  27.10628401],
       [-39.37493118,  33.74331058],
       [-32.83782235,  38.97676346],
       [ 34.27392917,  68.84014149],
       [ 21.15780217,  66.51522316],
       [ -4.37932365,  86.30759467],
       [  8.03473239,  59.11414878],
       [ 13.01838889,  72.95341865],
       [ 13.93605242,  73.02007091],
       [-10.70574046,  82.24083822],
       [ 17.91055628,  53.75594616],
       [ 21.59047521,  39.32463167],
       [ 32.70831989,  36.6118544 ],
       [ 12.98437777,  34.43894436],
       [ 23.69329337,  30.89465963]])